# Neptune as Graph Memory

TODO: what is mem0

TODO: types of mem0 storage [vector storage, graph storage]

References

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Connect to Neptune

To connect to AWS Neptune Analytics, you need to configure Neptune with your AWS profile details. I have configured AWS credentials in my home profile and can connect by specifying the AWS_PROFILE environment variable

TODO: Reference to: Neptune setup; aws-langchain

### 3. Configure OpenSearch

We're going to use OpenSearch as our vector store.  You can run [OpenSearch from docker image](https://docs.opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/):

```bash
docker pull opensearchproject/opensearch:3
```

And verify that it's running with a `<custom-admin-password>`:

```bash
 docker run -d -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=<custom-admin-password>" opensearchproject/opensearch:latest

 curl https://localhost:9200 -ku admin:<custom-admin-password>
```

We're going to connect [OpenSearch using the python client](https://github.com/opensearch-project/opensearch-py):

```bash
pip install "opensearch-py"
```

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [1]:
from mem0 import Memory
from opensearchpy import OpenSearch
from langchain_aws import NeptuneAnalyticsGraph
import os, logging, sys

logging.getLogger('mem0.memory.neptune_memory').setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig(
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)

Setup the Mem0 configuration using:
- openai as the embedder
- AWS Neptune Analytics instance as a graph store
- OpenSearch as the vector store

In [2]:
graph_identifier = os.environ.get("GRAPH_ID")
print(f"graph_identifier={graph_identifier}")
print(f"profile={os.environ.get("AWS_PROFILE")}")
opensearch_username = os.environ.get("OS_USERNAME")
opensearch_password = os.environ.get("OS_PASSWORD")
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": 1536
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "graph_identifier": graph_identifier,
            # "endpoint_url": endpoint_url,
            # "region": region,
        },
    },
    "vector_store": {
        "provider": "opensearch",
        "config": {
            "collection_name": "vector_store",
            "host": "localhost",
            "port": 9200,
            "user": opensearch_username,
            "password": opensearch_password,
            "use_ssl": False,
            "verify_certs": False,
        },
    },
}

graph_identifier=g-kwoidooqn7
profile=gamma


## Graph Memory initializiation

Initialize Memgraph as a Graph Memory store:

In [3]:
m = Memory.from_config(config_dict=config)

app_id = "movies"
user_id = "alice"

/Users/andrewc/git/acarbonetto/mem0/.venv/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()


## Store memories

Create memories:

In [4]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

Store memories in Neptune Analytics:

In [5]:
# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

DEBUG - Entity type map: {'movie': 'media', 'thriller': 'genre', 'sci-fi': 'genre', 'i': 'self', 'my': 'self'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'movie', 'entity_type': 'media'}, {'entity': 'thriller', 'entity_type': 'genre'}, {'entity': 'sci-fi', 'entity_type': 'genre'}]}}, {'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'I', 'entity_type': 'self'}, {'entity': 'my', 'entity_type': 'self'}]}}]}
DEBUG - Extracted entities: [{'source': 'alice', 'relationship': 'plans_to_watch', 'destination': 'movie'}, {'source': 'movie', 'relationship': 'is_a', 'destination': 'thriller'}, {'source': 'movie', 'relationship': 'is_a', 'destination': 'sci-fi'}, {'source': 'alice', 'relationship': 'does_not_like', 'destination': 'thriller'}, {'source': 'alice', 'relationship': 'likes', 'destination': 'sci-fi'}, {'source': 'movie', 'relationship': 'is_recommended_for', 'destination': 'alice'}]
DEBUG - _search_

## Search memories

In [6]:
for result in m.search("what does alice love?", user_id=user_id)["results"]:
    print(f"\"{result["memory"]}\" [score: {result["score"]}]")

DEBUG - Entity type map: {'alice': 'person'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'alice', 'entity_type': 'Person'}]}}]}
DEBUG - _search_graph_db
  query=
            MATCH (n)
            WHERE n.embedding IS NOT NULL AND n.user_id = $user_id
            WITH n, n.embedding AS embedding, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(
                n_embedding,
                n,
                {metric:"CosineSimilarity"}
            ) YIELD distance
            WITH n, round(2 * distance - 1) AS similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION
                MATCH (m)-[r]->(n) 
                RETURN m.name A

Neptune Analytics Graph

In [7]:
all_results = m.get_all(user_id=user_id)
for n in all_results['results']:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results['relations']:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - Retrieved 6 relationships
node "Planning to watch a movie tonight": [hash: bf55418607cfdca4afa311b5fd8496bd]
node "Not a big fan of thriller movies": [hash: 028dfab4483f28980e292f62578d3293]
node "Loves sci-fi movies": [hash: 1110b1af77367917ea2022355a16f187]
edge "alice" --plans_to_watch--> "movie"
edge "alice" --likes--> "sci-fi"
edge "movie" --is_a--> "sci-fi"
edge "alice" --does_not_like--> "thriller"
edge "movie" --is_a--> "thriller"
edge "movie" --is_recommended_for--> "alice"


In [8]:
neptune_graph = NeptuneAnalyticsGraph(graph_identifier)

query = """
        MATCH (n {user_id: $user_id})-[r]->(m {user_id: $user_id})
        RETURN n.name AS source, type(r) AS relationship, m.name AS target
        LIMIT $limit
        """
edge_results = neptune_graph.query(
    query, params={"user_id": user_id, "limit": 100}
)
print("----RELATIONSHIPS----")
for e in edge_results:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")


----RELATIONSHIPS----
edge "alice" --plans_to_watch--> "movie"
edge "alice" --likes--> "sci-fi"
edge "movie" --is_a--> "sci-fi"
edge "alice" --does_not_like--> "thriller"
edge "movie" --is_a--> "thriller"
edge "movie" --is_recommended_for--> "alice"


In [9]:
opensearch_index = "vector_store"
vector_store_config = config[opensearch_index]["config"]

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': vector_store_config["host"], 'port': vector_store_config["port"]}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (vector_store_config["user"], vector_store_config["password"]),
    use_ssl = vector_store_config["use_ssl"],
    verify_certs = vector_store_config["verify_certs"],
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

query = {"query":{"match_all": {}}}

response = client.search(
    body = query,
    index = opensearch_index,
)

print("----VECTORS----")
for v in response["hits"]["hits"]:
    print(f"vector id=\"{v["_source"]["id"]}\" data={v["_source"]["payload"]["data"]}")

----VECTORS----
vector id="51b37de7-f6a8-4fde-921d-12001ad7cba4" data=Planning to watch a movie tonight
vector id="1adffdc9-b01d-4a7f-a1c9-1684099a505a" data=Not a big fan of thriller movies
vector id="e7419edf-aab9-40b9-945c-1ee5dc2f9e6c" data=Loves sci-fi movies


In [10]:
m.reset()
# only works for neptune_memory
memory_graph = m.graph.reset()

WARNING - Resetting all memories
WARNING - Resetting index vector_store...
WARNING - Clearing graph...
